In [ ]:
'''
Imports.
'''
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math
import itertools

from scipy.stats import kstest
from scipy.stats import poisson
from scipy.stats import norm
from scipy.stats import t
from scipy.stats import chisquare

from statsmodels.stats.gof import gof_binning_discrete

from sklearn.preprocessing import StandardScaler
from sklearn.mixture import BayesianGaussianMixture
from sklearn import mixture


from scipy import linalg
import matplotlib as mpl
from collections import Counter

from matplotlib.colors import LogNorm
%matplotlib inline
plt.style.use('ggplot')
sns.set_style('whitegrid')

In [ ]:
'''
Load the calls data
Clean it a bit
'''
segment_size = 5000000
calls_data = pd.read_csv('DataFrames/calls.tsv', sep='\t')
calls_data['#CHR'] = pd.to_numeric(calls_data['#CHR'].str.replace('chr', ''))
calls_data = calls_data.sort_values(['CELL', '#CHR', 'START']).reset_index(drop=True)
calls_data['SEGMENT'] = ((calls_data['START'] > 0) & (calls_data['START'] % segment_size == 0)).cumsum()
calls_data

In [ ]:
'''
Functions
'''
# def spikiness(x):
#     if sum(x) == 0:
#         return np.nan
#     else:
#         return sum(abs(np.diff(x))) / sum(x)

def count_spikiness(x):
    return (sum(abs(x[1:].values - x[:-1].values)) / sum(x.values))

def rdr_spikiness(x):
    return 1/(sum(abs(x[1:].values - x[:-1].values)) / sum(x.values))

def mean_distance(x):
    st_vals = abs(x-x.mean())
    distance = 1/(1+sum(st_vals))
    return distance

def count_t(x):
    _centred = x - x.mean()
    return kstest(rvs=_centred, cdf='t', N=len(_centred), args=(1, ))[1]

def rdr_t(x):
    scaled_x = x * 100
    _centred = scaled_x - scaled_x.mean()
    return kstest(rvs=_centred, cdf='t', N=len(_centred), args=(1, ))[1]

def count_poisson(x):
    (obsfreq, expfreq, histg) = gof_binning_discrete(x, poisson, arg=(x.mean(),))
    return chisquare(obsfreq, expfreq, axis=None)[1]

def rdr_poisson(x):
    scaled_x = x * 100
    (obsfreq, expfreq, histg) = gof_binning_discrete(scaled_x, poisson, arg=(scaled_x.mean(),))
    return chisquare(obsfreq, expfreq, axis=None)[1]

def count_variance(x):
    _centred = x - x.mean()
    return _centred.var()

def rdr_variance(x):
    scaled_x = x * 100
    _centred = scaled_x - scaled_x.mean()
    return _centred.var()

count_poisson.__name__ = 'Poisson Score'
rdr_poisson.__name__ = 'Poisson Score'
count_t.__name__ = 'T Score'
rdr_t.__name__ = 'T Score'
mean_distance.__name__ = 'Distance metric'
count_variance.__name__ = 'Variance'
rdr_variance.__name__ = 'Variance'

In [ ]:
'''
P values per segment
'''
pvaldf = calls_data.groupby(['CELL', '#CHR', 'SEGMENT'], sort=False, as_index=False).agg({
    'COUNT': [count_t, count_poisson, count_variance, mean_distance],
    'RDR': [rdr_t, rdr_poisson, rdr_variance, mean_distance]
    })
pvaldf.to_csv('DataFrames/Segment_pvalues.csv')
# pvaldf = pd.read_csv('DataFrames/Segment_pvalues.csv', index_col=[0])
pvaldf

In [ ]:
'''
Remove np.inf, NaN and convert from multiindex
'''
pvaldf = pvaldf.replace([np.inf, -np.inf], np.nan)
print(f'NaN values:\n\n{pvaldf.isna().sum()}')
clean_pvaldf = pvaldf.dropna()
clean_pvaldf.columns = clean_pvaldf.columns.map(' '.join).str.strip()
clean_pvaldf.to_csv('DataFrames/Clean_Segment_pvalues.csv')
# clean_pvaldf = pd.read_csv('DataFrames/Clean_Segment_pvalues.csv', index_col=[0])
clean_pvaldf

In [ ]:
'''
Compute Spikiness
'''
spikiness_df = calls_data.groupby('CELL', sort=False, as_index=False).agg({'COUNT': count_spikiness, 'RDR': rdr_spikiness})
spikiness_df.to_csv('DataFrames/Spikiness.csv')
# spikiness_df = pd.read_csv('DataFrames/Spikiness.csv', index_col = [0])
spikiness_df

In [ ]:
'''
Bring everything together
'''
# combined_df = clean_pvaldf.groupby('CELL', sort=False, as_index=False).agg({column: 'median' for column in clean_pvaldf.columns.drop(['CELL', '#CHR', 'SEGMENT'])})
# combined_df['COUNT Spikiness'] = spikiness_df['COUNT'].values
# combined_df['RDR Spikiness'] = spikiness_df['RDR'].values
# combined_df.to_csv('DataFrames/Combined_pvalues.csv')
combined_df = pd.read_csv('DataFrames/Combined_pvalues.csv', index_col=[0])
combined_df


In [ ]:
'''
Plot Results
'''
fig = plt.figure(tight_layout=True, figsize=(16,16))
gs = fig.add_gridspec(4,12)
fig_ax1 = fig.add_subplot(gs[0, :3])
fig_ax2 = fig.add_subplot(gs[0, 3:6])
fig_ax3 = fig.add_subplot(gs[0, 6:9])
fig_ax4 = fig.add_subplot(gs[0, 9:12])
fig_ax5 = fig.add_subplot(gs[1, :3])
fig_ax6 = fig.add_subplot(gs[1, 3:6])
fig_ax7 = fig.add_subplot(gs[1, 6:9])
fig_ax8 = fig.add_subplot(gs[1, 9:12])
fig_ax9 = fig.add_subplot(gs[2, :3])
fig_ax10 = fig.add_subplot(gs[2, 3:6])
fig_ax11 = fig.add_subplot(gs[2, 6:9])
fig_ax12 = fig.add_subplot(gs[2, 9:12])
fig_ax13 = fig.add_subplot(gs[3, :3])
fig_ax14 = fig.add_subplot(gs[3, 3:6])
fig_ax15 = fig.add_subplot(gs[3, 6:9])
fig_ax16 = fig.add_subplot(gs[3, 9:12])

cph = sns.histplot(data=combined_df, x='COUNT Poisson Score', color='red', bins=50, ax=fig_ax1)
cpv = sns.scatterplot(data=combined_df, x='COUNT Variance', color='blue', y='COUNT Poisson Score', ax=fig_ax2)
rph = sns.histplot(data=combined_df, x='RDR Poisson Score', color='black', bins=50, ax=fig_ax3)
rpv = sns.scatterplot(data=combined_df, x='RDR Variance', y='RDR Poisson Score', color='orange', ax=fig_ax4)

cth = sns.histplot(data=combined_df, x='COUNT T Score', color='red', bins=50, ax=fig_ax5)
ctv = sns.scatterplot(data=combined_df, x='COUNT Variance', y='COUNT T Score', color='blue', ax=fig_ax6)
rth = sns.histplot(data=combined_df, x='RDR T Score',color='black', bins=50, ax=fig_ax7)
rtv = sns.scatterplot(data=combined_df, x='RDR Variance', y='RDR T Score', color='orange', ax=fig_ax8)

csh = sns.histplot(data=combined_df, x='COUNT Spikiness', color='red', bins=50, ax=fig_ax9)
css = sns.scatterplot(data=combined_df, x='COUNT Variance', y='COUNT Spikiness', color='blue', ax=fig_ax10)
csh = sns.histplot(data=combined_df, x='RDR Spikiness', bins=50, color='black', ax=fig_ax11)
css = sns.scatterplot(data=combined_df, x='RDR Variance', y='RDR Spikiness', color='orange', ax=fig_ax12)

a = sns.histplot(data=combined_df, x='COUNT Distance metric', color='red', bins=50, ax=fig_ax13)
b = sns.scatterplot(data=combined_df, x='COUNT Variance', y='COUNT Distance metric', color='blue', ax=fig_ax14)
c = sns.histplot(data=combined_df, x='RDR Distance metric', bins=50, color='black', ax=fig_ax15)
d = sns.scatterplot(data=combined_df, x='RDR Variance', y='RDR Distance metric', color='orange', ax=fig_ax16)

fig.savefig('Raw Figures/Methods_overall.pdf', format='pdf')

In [ ]:
'''
Look at these plots against total count number
'''
count_number_df = combined_df.copy()
total_counts = calls_data.groupby('CELL', sort=False, as_index=False).agg({'COUNT':'sum', 'RDR': 'sum'})
count_number_df['TOTAL COUNT'] = total_counts['COUNT']
count_number_df['TOTAL RDR'] = total_counts['RDR']



def _plot(axis, method):
    scatter = sns.scatterplot(data=count_number_df, x='TOTAL COUNT', y=method,color='black' if axis <=3 else 'orange', ax=axs[axis,0] if axis <=3 else axs[axis-4,1])
    scatter.set_ylim()
    scatter.set_yticks([])
    scatter.set_ylabel('')
    scatter.text(0.6, 0.8, f'{method}',fontsize=5,transform=scatter.transAxes)


methods = count_number_df.columns.drop(['CELL', 'COUNT Variance', 'RDR Variance', 'TOTAL COUNT', 'TOTAL RDR']).sort_values()
fig, axs = plt.subplots(int(len(methods)/2), 2, figsize=(20,20), tight_layout=True)
grphs = list(map(lambda method: _plot(*method), enumerate(methods)))
fig.savefig('Raw Figures/Total_count.pdf', format='pdf')

In [ ]:
'''
Look at cell plots for each
'''
plotting_df = combined_df.copy()
good_bad_dict = {x: [plotting_df.sort_values(x, ascending=False).head(1)['CELL'].values[0], plotting_df.sort_values(x, ascending=False).tail(1)['CELL'].values[0]] for x in plotting_df.columns.drop(['CELL', 'COUNT Variance', 'RDR Variance']).sort_values()}

gs_kw = dict(width_ratios=[2] * 2, height_ratios=[1] * 8)
figs, axes = plt.subplots(8, 2, constrained_layout=True, figsize=(70, 50), gridspec_kw=gs_kw, sharex='col', sharey='col')
figs.suptitle('Method Performance', fontsize='xx-large')
figs.set_constrained_layout_pads(w_pad=25/72, h_pad=25/72, hspace=0.05, wspace=0.05)

rnum = -1
for num, (method, cells) in enumerate(good_bad_dict.items()):
    for cell in cells:
        rnum += 1
        cell_df = calls_data[calls_data['CELL'] == cell].copy()
        cell_df['Position'] = cell_df['START'].cumsum()
        s = sns.scatterplot(data=cell_df, x='Position', y='COUNT' if num <= 3 else 'RDR',color='orange' if num <= 3 else 'black',legend=False, ax=axes[rnum, 0] if num <= 3 else axes[rnum - 8, 1])
        s.set_title(f'BEST - method: {method}' if rnum % 2 == 0 else f'WORST - method: {method}')
        s.set_xlim(0, cell_df['Position'].max())
        s.set_ylim((0, 150) if num <= 3 else (None, None))
        s.text(0.8, 1.1, f'Variance: {plotting_df[plotting_df["CELL"] == cell]["COUNT Variance"].values}' if num <= 3 else f'Variance: {plotting_df[plotting_df["CELL"] == cell]["RDR Variance"].values}', bbox=dict(facecolor='red', alpha=0.5), transform=s.transAxes)
        vlines = list(map(lambda x: s.axvline(x), cell_df.drop_duplicates('#CHR', keep='last')['Position']))

figs.savefig('Raw Figures/Performance.pdf', format='pdf')


In [ ]:
'''
Plot boxplots for all metrics for clone and no clone cells
'''
mappings_data = pd.read_csv('DataFrames/mapping.tsv', sep='\t')
clone_df = combined_df.copy()
clone_df['Clone'] = clone_df['CELL'].isin(mappings_data[mappings_data['CLONE'] != 'None']['#CELL'].reset_index(drop=True))
clone_df = clone_df.replace([True, False], ['Has clone', 'No clone'])

methods = clone_df.columns.drop(['CELL', 'Clone']).sort_values()

fig, axs = plt.subplots(len(methods), figsize=(25,100), tight_layout=True, sharex=True)
boxplots = list(map(lambda enum: _plot(enum[0], enum[1]), enumerate(methods)))
fig.savefig('Raw Figures/Boxplots.pdf', format='pdf')

In [ ]:
'''
Take example cells and plot them


rdr = count bin/count nm bin x mn total counts/total counts
'''
random_cell_plot_df = combined_df.copy()
random_cell_choice = random.choice(random_cell_plot_df['CELL'].unique())
cellcalls = calls_data[calls_data['CELL'] == random_cell_choice].copy()
cellcalls['TOTCOUNT'] = cellcalls['START'].cumsum()
metrics = random_cell_plot_df[random_cell_plot_df['CELL'] == random_cell_choice]
print(f'Average counts: {cellcalls["COUNT"].mean()}\t Variance counts: {cellcalls["COUNT"].var()}')
print(f'Average rdr: {cellcalls["RDR"].mean()}\t Variance rdr: {cellcalls["RDR"].var()}')
print(f'T:{metrics["COUNT T Score"].values[0]}')

fig, ax = plt.subplots(2,1,tight_layout=True, figsize=(30,20))
scatp = sns.scatterplot(data=cellcalls, x='TOTCOUNT', y='COUNT', ax=ax[0])
scatp.set_ylim(0,200)
scatr = sns.scatterplot(data=cellcalls, x='TOTCOUNT', y='RDR', ax=ax[1])

In [ ]:
'''clustering'''
cell_segment_df = calls_data[(calls_data['CELL'] == 'AAACCTGGTACCAGTT') & (calls_data['SEGMENT'] == 1167)].copy()
count_array = cell_segment_df['RDR'].values.reshape(-1, 1)
start_array = cell_segment_df['START'].values.reshape(-1, 1)
_X = np.concatenate((start_array, count_array), axis=1)
gmm = BayesianGaussianMixture(n_components=10).fit(count_array)

color_iter = itertools.cycle(['navy', 'c', 'cornflowerblue', 'gold', 'darkorange'])
fig, ax = plt.subplots(figsize=(16,10))
def plot_results(X, Y_, means, covariances, index, title):
    splot = plt.subplot(2, 1, 1 + index)
    for i, (mean, covar, color) in enumerate(zip(
            means, covariances, color_iter)):
        v, w = linalg.eigh(covar)
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])
        if not np.any(Y_ == i):
            continue
        plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], 5, color='black')
        angle = np.arctan(u[1] / u[0])
        angle = 180. * angle / np.pi
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        splot.add_artist(ell)
    plt.xticks(())
    plt.yticks(())
    plt.title(title)

dpgmm = mixture.BayesianGaussianMixture(n_components=10, covariance_type='full', weight_concentration_prior=0.01, weight_concentration_prior_type='dirichlet_distribution',mean_precision_prior=1e-2, max_iter=100).fit(_X)
plot_results(_X, dpgmm.predict(_X), dpgmm.means_, dpgmm.covariances_, 1, '')

fig.savefig('Thesis Figures/clustering.pdf', format='pdf')

In [ ]:
'''clustering'''
def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=35, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=2, linewidths=12,
                color=cross_color, zorder=11, alpha=1)

def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,
                linewidths=2, colors='r', linestyles='dashed')
    
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("$x_1$", fontsize=14)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
'''clustering'''
plt.figure(figsize=(8, 4))
plot_gaussian_mixture(dpgmm, _X)
plt.savefig('Thesis Figures/clustering2.pdf', format='pdf')

In [ ]:
'''
clustering percent function
'''
def segment_clustering(series):
    num_components = 10
    if len(series) < num_components:
        return np.nan
    array = series.values.reshape(-1,1)
    number_of_clusters = BayesianGaussianMixture(n_components=num_components, covariance_type='full', weight_concentration_prior=0.01, weight_concentration_prior_type='dirichlet_distribution', mean_precision_prior=1e-2, n_init=3,max_iter=100).fit_predict(array)
    most_common_cluster = np.bincount(number_of_clusters).argmax()
    deviant_assignments = np.count_nonzero(number_of_clusters != most_common_cluster)
    return deviant_assignments

In [ ]:
'''pre clustering - segment sampling'''
number_of_samples = 50

pre_cluster = calls_data.copy()

segment_lst = list(itertools.chain(*[random.sample(range(sub['SEGMENT'].min(), sub['SEGMENT'].max()), number_of_samples) for cell, sub in pre_cluster.groupby('CELL', sort=False)]))

precluster_df = pre_cluster[pre_cluster['SEGMENT'].isin(segment_lst)].reset_index(drop=True)
precluster_df

In [ ]:
'''clustering'''
# cluster_df = precluster_df.groupby(['CELL', 'SEGMENT'], sort=False, as_index=False).agg({'COUNT': segment_clustering, 'RDR': segment_clustering})
# cluster_df.to_csv('DataFrames/clustering_df_50samples.csv')
cluster_df = pd.read_csv('DataFrames/clustering_df_50samples.csv')
cluster_df

In [ ]:
total_cluster_cell = cluster_df.groupby('CELL', sort=False, as_index=False).agg({'COUNT': 'sum', 'RDR': 'sum'})
cell_total_bin_lst = list(map(lambda x: len(precluster_df[precluster_df['CELL'] == x]), total_cluster_cell['CELL'].unique()))
total_cluster_cell['TOTAL BINS'] = cell_total_bin_lst
total_cluster_cell


In [ ]:
total_cluster_cell['COUNT PERCENTAGE'] = (total_cluster_cell['COUNT'] / total_cluster_cell['TOTAL BINS']) * 100
total_cluster_cell['RDR PERCENTAGE'] = (total_cluster_cell['RDR'] / total_cluster_cell['TOTAL BINS']) * 100
total_cluster_cell

In [ ]:
fig, ax = plt.subplots(1,2,tight_layout=True, sharey=True)
s = sns.histplot(data=total_cluster_cell, x='COUNT PERCENTAGE', bins=50, color='red', ax=ax[0])
s.set_title('Percentages for read counts');s.set_xlabel('')
sr = sns.histplot(data=total_cluster_cell, x='RDR PERCENTAGE', bins=50, color='black', ax=ax[1])
sr.set_title('Percentages for RDR');sr.set_xlabel('')
fig.savefig('Thesis Figures/percentage_distribution.pdf', format='pdf')

In [ ]:
data_oi = total_cluster_cell[(total_cluster_cell['RDR PERCENTAGE'] > 9) & (total_cluster_cell['RDR PERCENTAGE'] < 10)]
data_oi

In [ ]:
cells_oi = data_oi['CELL'].unique()
count_t_scores = combined_df[combined_df['CELL'].isin(cells_oi)]

fig, ax = plt.subplots(2,2, tight_layout=True, sharex='col', sharey='row', figsize=(12,12))
s1 = sns.boxplot(data=count_t_scores, color='red',x='COUNT T Score', ax=ax[0,0])
s2 = sns.boxplot(data=count_t_scores, color='black',x='RDR T Score', ax=ax[0,1])
s1.set_title('Read counts')
s2.set_title('RDR')
s1.set_ylabel('')
s2.set_ylabel('')
h1 = sns.histplot(data=combined_df, x='COUNT T Score', color='red', bins=50, ax=ax[1,0])
h2 = sns.histplot(data=combined_df, x='RDR T Score', color='black', bins=50, ax=ax[1,1])
h1.axvline(count_t_scores['COUNT T Score'].median(), color='green', label='Threshold')
h2.axvline(count_t_scores['RDR T Score'].median(), color='green', label='Threshold')
h1.legend()
h2.legend()
fig.suptitle('Boxplots for reference cells used to infer cutoff for all cells', fontsize=16)

fig.savefig('Thesis Figures/Boxplot_threshold.pdf')

In [ ]:
(len(combined_df[combined_df['COUNT T Score'] > count_t_scores['COUNT T Score'].median()]) / 1829) * 100


In [ ]:
(len(combined_df[combined_df['RDR T Score'] > count_t_scores['RDR T Score'].median()]) / 1829) * 100

In [ ]:
good_cells = combined_df[combined_df['RDR T Score'] > count_t_scores['RDR T Score'].median()]['CELL'].values

calls_data['QUALITY'] = calls_data['CELL'].isin(good_cells)
calls_data = calls_data.replace(True, 'Good')
calls_data = calls_data.replace(False, 'Bad')
calls_data

In [ ]:
combined_df['QUALITY'] = combined_df['CELL'].isin(good_cells)
combined_df = combined_df.replace([True, False], ['Good', 'Bad'])
combined_df

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
f = sns.boxplot(data=combined_df, x='QUALITY', y='RDR Variance')
f.set_ylabel('Variance')
fig.savefig('Thesis Figures/GoodBadBoxplot.pdf', format='pdf')

In [ ]:
mappings_data = pd.read_csv('DataFrames/mapping.tsv', sep='\t')
cells_with_clone = mappings_data[mappings_data['CLONE'] != 'None']['#CELL'].values
combined_df['CLONE'] = combined_df['CELL'].isin(cells_with_clone)
combined_df = combined_df.replace([True, False], ['Has Clone', 'No Clone'])
combined_df

In [ ]:
x = combined_df[combined_df['CLONE'] == 'No Clone'].copy()
xb = combined_df[(combined_df['CLONE'] == 'No Clone') & (combined_df['QUALITY'] == 'Bad')].copy()

In [ ]:
percent_bad = (len(xb) / len(x)) * 100
percent_bad